# Day 4: Giant Squid

## Part 1

You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine.

Maybe it wants to play bingo?

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

The submarine has a bingo subsystem to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:

```
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
8  2 23  4 24
21  9 14 16  7
6 10  3 18  5
1 12 20 15 19

3 15  0  2 22
9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
2  0 12  3  7
```

After the first five numbers are drawn (7, 4, 9, 5, and 11), there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):

```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```

After the next six numbers are drawn (17, 23, 2, 0, 14, and 21), there are still no winners:

```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```

Finally, 24 is drawn:

```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```

At this point, the third board wins because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: 14 21 17 24 4).

The score of the winning board can now be calculated. Start by finding the sum of all unmarked numbers on that board; in this case, the sum is 188. Then, multiply that sum by the number that was just called when the board won, 24, to get the final score, 188 * 24 = 4512.

To guarantee victory against the giant squid, figure out which board will win first. What will your final score be if you choose that board?

In [1]:
with open('input/day4/input.txt', 'r') as finput:
    raw_input = finput.readlines()

In [2]:
class Board:
    class BoardNum:
        def __init__(self, number):
            self.number = number
            self.found = False

        def __repr__(self):
            return f"Number: {self.number} - Found: {self.found}"

    def __init__(self):
        self.board = []  # type: [[Board.BoardNum]]
        self.has_bingo = False
        self.last_stamp = -1

    def add_row(self, row: str):
        self.board.append([self.BoardNum(item)
                           for item in row.split()])

    def _check_for_bingo(self):
        for row in self.board:
            all_found = True
            for col in row:  # type: Board.BoardNum
                if not col.found:
                    all_found = False

            if all_found:
                self.has_bingo = True
                return True

        for idx in range(len(self.board)):
            all_found = True
            for idy in range(len(self.board)):
                if not self.board[idy][idx].found:
                    all_found = False

            if all_found:
                self.has_bingo = True
                return True

        return False

    def stamp(self, num: int) -> bool:
        if self.has_bingo:
            raise ValueError("uhhh")
        self.last_stamp = num
        for row in self.board:  # type: [Board.BoardNum]
            for col in row:  # type: Board.BoardNum
                if col.number == num:
                    col.found = True
                    return self._check_for_bingo()
        return False

    def get_score(self):
        acc = 0
        for row in self.board:  # type: [Board.BoardNum]
            for col in row:  # type: Board.BoardNum
                if not col.found:
                    acc += int(col.number)

        return acc * int(self.last_stamp)

    def __repr__(self):
        return f"Has Bingo: {self.has_bingo}"


class SubBingo:

    def __init__(self):
        self.boards = list()  # type: [Board]
        self.winners = list()

    def add_boards(self, board_list: list):
        board = None
        for line in board_list:
            if not line:
                # empty line means new board starting
                if board:
                    self.boards.append(board)

                board = Board()
            else:
                board.add_row(line)

        if board.board:  # if we ran out of input, and had a board in process (no empty line after), save it
            self.boards.append(board)

    def play_bingo(self, cmds: str, cont_after_win: bool=False):
        for num in cmds.split(','):
            for idx, board in enumerate(self.boards):
                if not board.has_bingo:
                    if board.stamp(num):
                        print(f"Bingo on {idx}! Their score was {board.get_score()}")
                        self.winners.append((board, board.get_score))
                        if not cont_after_win:
                            return

    def __repr__(self):
        return f"Number of boards: {len(self.boards)}"

In [3]:
# line 1 is the bingo game string, the rest are boards
commands = raw_input[0]
boards = [x.strip() for x in raw_input[1:]]

In [4]:
bingo = SubBingo()
bingo.add_boards(boards)

In [5]:
bingo.play_bingo(commands)

Bingo on 41! Their score was 89001


## Part Two

On the other hand, it might be wise to try a different strategy: let the giant squid win.

You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to figure out which board will win last and choose that one. That way, no matter which boards it picks, it will win for sure.

In the above example, the second board is the last to win, which happens after 13 is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to 148 for a final score of 148 * 13 = 1924.

Figure out which board will win last. Once it wins, what would its final score be?


In [6]:
p2bingo = SubBingo()
p2bingo.add_boards(boards)

p2bingo.play_bingo(commands, cont_after_win=True)

Bingo on 41! Their score was 89001
Bingo on 54! Their score was 42630
Bingo on 1! Their score was 45122
Bingo on 68! Their score was 71111
Bingo on 72! Their score was 48510
Bingo on 78! Their score was 19840
Bingo on 86! Their score was 21450
Bingo on 92! Their score was 71616
Bingo on 32! Their score was 65455
Bingo on 37! Their score was 62700
Bingo on 85! Their score was 62700
Bingo on 21! Their score was 26624
Bingo on 53! Their score was 39520
Bingo on 59! Their score was 35646
Bingo on 31! Their score was 18240
Bingo on 47! Their score was 17600
Bingo on 24! Their score was 28782
Bingo on 82! Their score was 26784
Bingo on 94! Their score was 26676
Bingo on 0! Their score was 44184
Bingo on 9! Their score was 28140
Bingo on 17! Their score was 24696
Bingo on 62! Their score was 28980
Bingo on 65! Their score was 51912
Bingo on 87! Their score was 32760
Bingo on 57! Their score was 27730
Bingo on 76! Their score was 32686
Bingo on 77! Their score was 31506
Bingo on 89! Their scor